# Preprocess

## What you learn

Datasetを機械学習モデルが認識しやすい形に変換し、Preprocessed Datasetを作成します。必要に応じデータ同士を組み合わせて新しいデータを作るFeature Engineeringを行います。

## Why you need

人間に理解しやすいデータと機械学習モデルにとって理解しやすいデータは異なります。また、テキストや文字データはそのままでは機械学習モデルに入力できないため、数値表現に変換する必要があります。きちんとした定義はありませんが、データの欠損や外れ値の除去などネガティブな要素を低減するための処理を前処理、データの特徴をより強調したり学習しやすい形式に変換する処理をFeature Engineeringと呼びます。

## How to do

データには画像、テキスト、音声、数値、カテゴリ項目など様々な種類があり、データの種類ごとに前処理、Feature Engineeringの手法は異なります。本性は基本的な数値とカテゴリ項目の前処理についてTechnical Exerciseを示します。まず、データを読み込みます。

In [1]:
from pathlib import Path
import pandas as pd


raw_data_path = Path("../data/raw/house_prices.csv")
house_price_df = pd.read_csv(raw_data_path, na_values="?")

### Technical Exercise

数値項目の代表的な前処理は標準化です。数値項目の値を平均0、分散1に値をそろえる処理です。項目によって値の範囲がばらばらである時、標準化を行うことで学習がしやすくなります。

数値項目の標準化は`scikit-learn`というライブラリの`StandardScaler`で行えます。`SalePrice`を`StandardScaler`で標準化してみます。標準化前の`StandardScaler`の統計量は次の通りです。

In [2]:
house_price_df["SalePrice"].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

`StandardScaler`で標準化してみましょう。

In [3]:
from sklearn.preprocessing import StandardScaler


standard_scaler = StandardScaler()
standard_scaler.fit(house_price_df["SalePrice"].values.reshape((-1, 1)))
scaled_value = standard_scaler.transform(house_price_df["SalePrice"].values.reshape((-1, 1)))
pd.DataFrame(scaled_value).describe()

,0
count,1.460000e+03
mean,1.362685e-16
std,1.000343e+00
min,-1.838704e+00
25%,-6.415162e-01
50%,-2.256643e-01
75%,4.165294e-01
max,7.228819e+00


平均(mean)は0、標準偏差は1に近くなっていることがわかります。

ドロップダウンリストやラジオボタンから選択するようなカテゴリ項目は数値に変換する必要があります。方法は2つあります。1つ目は、単純に項目ごと上から順に番号を割り当てる手法です。`scikit-learn`では`OrdinalEncoder`で実装できます。`ExterCond`を`OrdinalEncoder`で数値にしてみましょう。

In [4]:
house_price_df["ExterCond"].value_counts()

TA    1282
Gd     146
Fa      28
Ex       3
Po       1
Name: ExterCond, dtype: int64

In [5]:
from sklearn.preprocessing import OrdinalEncoder


ordinal_encoder = OrdinalEncoder()
ordinal_encoder.fit(house_price_df["ExterCond"].values.reshape((-1, 1)))
ordinal_encoded_category = ordinal_encoder.transform(house_price_df["ExterCond"].values.reshape((-1, 1)))
pd.Series(ordinal_encoded_category.flatten()).value_counts()

4.0    1282
2.0     146
1.0      28
0.0       3
3.0       1
dtype: int64

数値に変換できていることがわかります。ただ、数値にすることで項目間に順列性が発生します。日本、アメリカ、イギリス。。。という項目があったとして、これを数値に変換すると0,1,2となり日本<アメリカという数値上の関係が発生してしまいます。順列関係が好ましくない場合は、2つ目の方法として値ごとに項目を分離します。「日本」「アメリカ」「イギリス」という項目を作って、チェックがあれば1、それ以外は0とします。`scikit-learn`では`OnehotEncoding`で実装できます。

In [6]:
from sklearn.preprocessing import OneHotEncoder


one_hot_encoder = OneHotEncoder(handle_unknown="ignore")
one_hot_encoder.fit(house_price_df["ExterCond"].values.reshape((-1, 1)))
encoded_category = one_hot_encoder.transform(house_price_df["ExterCond"].values.reshape((-1, 1)))
pd.DataFrame(encoded_category.toarray().astype("int64"), columns=one_hot_encoder.categories_)

,Ex,Fa,Gd,Po,TA
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1
...,...,...,...,...,...
1455,0,0,0,0,1
1456,0,0,0,0,1
1457,0,0,1,0,0
1458,0,0,0,0,1


どの列にどのような前処理をするかは、ヒアリングにより明らかにしていく必要があります。今回は、一律にカテゴリの項目には`OrdinalEncoder`、数値の項目には'StandardScaler'を適用することとします。

In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer


# N/Aの割合が高い項目を除外
formatted_df = house_price_df.drop(columns=["Id", "PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu"], axis=1)

# 70万ドルを超える高額住宅を除外
formatted_df = formatted_df[formatted_df["SalePrice"] < 700000]

numerical_columns = formatted_df.select_dtypes("number").columns
categorical_columns = formatted_df.columns.difference(numerical_columns)
numerical_columns = numerical_columns.drop("SalePrice")
encoder = make_column_transformer(
        (StandardScaler(), numerical_columns),
        (StandardScaler(), ["SalePrice"]),
        (OrdinalEncoder(), categorical_columns),
)

preprocessed = encoder.fit_transform(formatted_df)
preprocessed_df = pd.DataFrame(preprocessed, columns=(numerical_columns.tolist() + ["SalePrice"] + categorical_columns.tolist()))
preprocessed_df = preprocessed_df.fillna(preprocessed_df.median())

前処理済みのデータは次のようになります。

In [8]:
preprocessed_df.head(5)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,MSZoning,MasVnrType,Neighborhood,PavedDrive,RoofMatl,RoofStyle,SaleCondition,SaleType,Street,Utilities
0,0.073426,-0.204952,-0.205997,0.658506,-0.517011,1.051798,0.878933,0.519640,0.582653,-0.288867,...,3.0,1.0,5.0,2.0,1.0,1.0,4.0,8.0,1.0,0.0
1,-0.871868,0.416540,-0.090762,-0.068293,2.178350,0.157771,-0.428655,-0.575767,1.182537,-0.288867,...,3.0,2.0,24.0,2.0,1.0,1.0,4.0,8.0,1.0,0.0
2,0.073426,-0.080654,0.074575,0.658506,-0.517011,0.985574,0.830504,0.329620,0.097453,-0.288867,...,3.0,1.0,5.0,2.0,1.0,1.0,4.0,8.0,1.0,0.0
3,0.309749,-0.412116,-0.095772,0.658506,-0.517011,-1.862068,-0.719230,-0.575767,-0.498020,-0.288867,...,3.0,2.0,6.0,2.0,1.0,1.0,0.0,8.0,1.0,0.0
4,0.073426,0.582271,0.376190,1.385305,-0.517011,0.952461,0.733645,1.380317,0.470175,-0.288867,...,3.0,1.0,15.0,2.0,1.0,1.0,4.0,8.0,1.0,0.0


前処理に加えて、データを加工したり組み合わせるFeature Engineeringを行うことで、データの傾向を顕著に表す特徴を作成する場合もあります。詳細はReferenceを参照してください。

Preprocessの終わりに、前処理済みのデータをTrainで使用する学習用データ、Testで使用する評価用データに分割して保存します。

In [9]:
from sklearn.model_selection import train_test_split


train_df, test_df = train_test_split(preprocessed_df, test_size=0.2) 

In [10]:
train_df.to_csv(Path("../data/processed/train.csv"), index=False)
test_df.to_csv(Path("../data/processed/test.csv"), index=False)

評価を行う際は、前処理により変形したデータを元に戻す必要があるため前処理のプロセスも保存をします。

In [11]:
from joblib import dump


encoder_path = Path("../model/encoder.joblib")
dump(encoder, encoder_path) 

['../model/encoder.joblib']

### Communication Exercise

Data ScientistとしてPreprocessを行うにあたり、誰にどのような質問をする必要がありますか?

* Product Manager
* Business Analyst
* Data Analyst
* Architect 
* DevOps Engineer
* Software Engineer
* Operator
* System Admin
* IT Auditor
* Data architect
* Domain Expert

#### Example

* Data Analyst
   * 現在分析を行うにどのような前処理をしていますか。

## Next Step

[Train](05_train.ipynb)

## References

1. 本橋 智光 and 株式会社ホクソエム. [前処理大全［データ分析のためのSQL/R/Python実践テクニック］](https://www.amazon.co.jp/dp/B07C3JFK3V/ref=dp-kindle-redirect?_encoding=UTF8&btkr=1). 2018.
2. Alice Zheng, Amanda Casari and 株式会社ホクソエム. [機械学習のための特徴量エンジニアリング ―その原理とPythonによる実践](https://www.amazon.co.jp/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%81%AE%E3%81%9F%E3%82%81%E3%81%AE%E7%89%B9%E5%BE%B4%E9%87%8F%E3%82%A8%E3%83%B3%E3%82%B8%E3%83%8B%E3%82%A2%E3%83%AA%E3%83%B3%E3%82%B0-%E2%80%95%E3%81%9D%E3%81%AE%E5%8E%9F%E7%90%86%E3%81%A8Python%E3%81%AB%E3%82%88%E3%82%8B%E5%AE%9F%E8%B7%B5-%E3%82%AA%E3%83%A9%E3%82%A4%E3%83%AA%E3%83%BC%E3%83%BB%E3%82%B8%E3%83%A3%E3%83%91%E3%83%B3-Alice-Zheng/dp/4873118689). 2019. 
3. Henrik Brink, Joseph Richards and Mark Fetherolf [Real-World Machine Learning](https://www.amazon.co.jp/dp/B097834SKF/ref=dp-kindle-redirect?_encoding=UTF8&btkr=1). 2016.

関連するAWSのサービス

1. Data Wrangler
2. SageMaker Processing
